In [0]:
AWS_SECRET_ACCESS_KEY = dbutils.secrets.get(
    scope="awscredentials",
    key="awssecret"
)
AWS_ACCESS_KEY_ID = dbutils.secrets.get(
    scope="awscredentials",
    key="awskey"
)

spark.conf.set("fs.s3a.access.key", AWS_ACCESS_KEY_ID)
spark.conf.set("fs.s3a.secret.key", AWS_SECRET_ACCESS_KEY)



In [0]:
from pyspark.sql.types import StructType, StructField, StringType

In [0]:
df_schema = StructType([
    StructField("comment_count", StringType(), True),
    StructField("duration", StringType(), True),
    StructField("like_count", StringType(), True),
    StructField("published_at", StringType(), True),
    StructField("title", StringType(), True),
    StructField("video_id", StringType(), True),
    StructField("view_count", StringType(), True)
])

In [0]:
dbutils.widgets.text("p_file_name", "")
v_file_name = dbutils.widgets.get("p_file_name")


In [0]:
df = spark.read.schema(df_schema).option("multiline", "true").json(f"s3a://youtubeetl852147/{v_file_name}")


In [0]:
from pyspark.sql.functions import current_timestamp
df_imported_date = df.withColumn("created_date", current_timestamp())

In [0]:
spark.sql("CREATE DATABASE IF NOT EXISTS yt_vids_data")

save_path = "abfss://raw@youtubevideosmetadata951.dfs.core.windows.net/yt_vids_data"

(df_imported_date.write
  .format("delta")                            
  .mode("append")                          
  .option("path", save_path)
  .saveAsTable("yt_vids_data.raw")     
)